In [303]:

import pandas as pd
import random
import datetime

In [304]:

dye_names = [ "8E","P","M","D","H"]
work_days = 5
week_start_date = datetime.datetime(2021, 8, 30)
capacity_per_day = {
"8E":288000,
"P":57600,
"M":100800,
"D":72000,
"H":86400,
}
week_days = list(
    week_start_date + datetime.timedelta(days=i) for i in range(work_days)
)


In [305]:
def random_requirement_generator(size: int) -> dict:
    for i in range(size):
        yield {
            "part_id": i+1,
            "dye":random.choice(dye_names),
            "cust": random.choice(["Autoliv","KSS","TRW"]),
            "safety_stock":0,
            "cards":random.randint(0,5),
            "cards_quantity":random.randint(0,14401),
        }
        
requirments = list(req for req in random_requirement_generator(100))

In [306]:
requirements_df = pd.DataFrame.from_records(data=requirments)

requirements_df["build_quantity"] = list(
    row["cards"]*row["cards_quantity"] for index, row in requirements_df.iterrows() 
)
requirements_df["dye_capacity"] = list (
    capacity_per_day[row["dye"]]*work_days for index, row in requirements_df.iterrows()
)

total_build_quantity = requirements_df.groupby(by=requirements_df["dye"]) \
                        .sum() \
                        .reset_index() \
                        .drop(
                            columns=["part_id","safety_stock","cards","cards_quantity","dye_capacity"]
                        ) \
                        .rename({
                            "build_quantity":"total_build_qty"
                        })


In [307]:

total_build_quantity['capacity_per_day'] = list(capacity_per_day[row["dye"]] for index, row in total_build_quantity.iterrows())
total_build_quantity['capacity_per_week'] = list(capacity_per_day[row["dye"]]*work_days for index, row in total_build_quantity.iterrows())
total_build_quantity

,dye,build_quantity,capacity_per_day,capacity_per_week
0,8E,528826,288000,1440000
1,D,554562,72000,360000
2,H,183014,86400,432000
3,M,472534,100800,504000
4,P,366300,57600,288000


In [308]:
requirements_df

,part_id,dye,cust,safety_stock,cards,cards_quantity,build_quantity,dye_capacity
0,1,M,KSS,0,2,7043,14086,504000
1,2,P,Autoliv,0,4,10851,43404,288000
2,3,P,TRW,0,5,14085,70425,288000
3,4,D,Autoliv,0,0,3375,0,360000
4,5,D,KSS,0,5,7196,35980,360000
...,...,...,...,...,...,...,...,...
95,96,M,TRW,0,5,7922,39610,504000
96,97,P,TRW,0,5,176,880,288000
97,98,P,KSS,0,0,468,0,288000
98,99,8E,KSS,0,3,4811,14433,1440000


In [309]:
end = requirements_df.loc[requirements_df['dye'] == '8E']
princess = requirements_df.loc[requirements_df['dye'] == 'P']
maggie = requirements_df.loc[requirements_df['dye'] == 'M']
dyeHard = requirements_df.loc[requirements_df['dye'] == 'D']
high5 = requirements_df.loc[requirements_df['dye'] == 'H']


In [310]:

def scheduler(df):
    return pd.DataFrame.from_records(data=list(record for record in date_record_generator(df)))

In [311]:
def date_record_generator(df):
    i=0
    sum=0
    for index, rows in df.iterrows():
        sum += rows["build_quantity"]
        if sum > capacity_per_day[rows["dye"]]:
            difference = sum-capacity_per_day[rows["dye"]]
            split = rows["build_quantity"] - difference
            yield {
            "index":index,
            "Date": week_days[i],
            "part_id" : rows["part_id"],
            "dye": rows["dye"],
            "cards": rows["cards"],
            "cards_quantity": rows["cards_quantity"],
            "build_quantity": split
            }
            i += 1
            sum = difference
            yield {
            "index":index,
            "Date": week_days[i],
            "part_id" : rows["part_id"],
            "dye": rows["dye"],
            "cards": rows["cards"],
            "cards_quantity": rows["cards_quantity"],
            "build_quantity": difference
            }
        else:
            yield {
            "index":index,
            "Date": week_days[i],
            "part_id" : rows["part_id"],
            "dye": rows["dye"],
            "cards": rows["cards"],
            "cards_quantity": rows["cards_quantity"],
            "build_quantity": rows["build_quantity"]
            }

In [312]:
capacity_per_day

{'8E': 288000, 'P': 57600, 'M': 100800, 'D': 72000, 'H': 86400}

In [313]:
End_scheduled = scheduler(end)
End_scheduled = pd.DataFrame.from_records(data=End_scheduled)
End_scheduled

,index,Date,part_id,dye,cards,cards_quantity,build_quantity
0,10,2021-08-30,11,8E,3,10037,30111
1,11,2021-08-30,12,8E,1,1730,1730
2,18,2021-08-30,19,8E,2,8886,17772
3,24,2021-08-30,25,8E,3,7148,21444
4,27,2021-08-30,28,8E,1,2271,2271
5,30,2021-08-30,31,8E,4,12825,51300
6,33,2021-08-30,34,8E,3,993,2979
7,35,2021-08-30,36,8E,5,10516,52580
8,40,2021-08-30,41,8E,2,10468,20936
9,43,2021-08-30,44,8E,3,6503,19509
